In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
path = SparkFiles.get("home_sales_revised.csv")
df = spark.read.csv(path, sep=",", header=True, inferSchema=True)

In [4]:
# 2. Create a temporary view of the DataFrame.
df.show()
df.createOrReplaceTempView("home_sales")

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import month, year
spark.sql("select year(home_sales.date), round(avg(price),2) as Avg from home_sales where bedrooms = 4 group by year(date) sort by year(date)").show()

+----------+---------+
|year(date)|      Avg|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("select date_built, round(avg(price),2) as Avg_2 from home_sales where bedrooms = 3 and bathrooms = 3 group by date_built sort by date_built").show()

+----------+---------+
|date_built|    Avg_2|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("select date_built, round(avg(price),2) as Avg_3 from home_sales where bedrooms = 3 and bathrooms = 3 and floors = 2 and sqft_living >= 2000 group by date_built sort by date_built").show()


+----------+---------+
|date_built|    Avg_3|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("select view, round(avg(price),2) as Avg_4 from home_sales where price >= 350000 group by view sort by view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|    Avg_4|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 0.3558049201965332 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("select view, round(avg(price),2) as Avg_5 from home_sales where price >= 350000 group by view sort by view").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|    Avg_5|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 0.18027734756469727 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.parquet('parquet_home_sales', mode='overwrite')
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

Py4JJavaError: An error occurred while calling o52.parquet.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputComm

In [ ]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('home_sales_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql("select view, round(avg(price),2) as Avg_6 from p_home_sales where price >= 350000 group by view sort by view").show()

print("--- %s seconds ---" % (time.time() - start_time))

In [13]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [14]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False